In [ ]:
import json
import pandas as pd

images = []
anns = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()
class_num = 11

def make_img(imgph):
    answer=dict()
    answer['license']=0
    answer['url']=None
    answer['file_name']=imgph
    answer['height']=512
    answer['width']=512
    answer['date_captured']=None
    answer['id']=9999
    return answer

def make_ann(img,annin):
    answers = []
    for j in range(class_num):
        tempanns = []
        if len(annin[0][j]) > 0:
            for o in annin[0][j]:
                ann = dict()
                ann['id'] = 9999
                ann['image_id'] = img['id']
                ann['category_id'] = str(j)
                ann['segmentation'] = []
                ann['area'] = 0
                ann['bbox'] = o[0:4]
                ann['iscrowd'] = 0
                tempanns.append(ann)
        if len(annin[1][j]) > 0:
            for idx, o in enumerate(annin[1][j]):
                templist = []
                if len(o) != 512:
                    for oo in o:
                        mask = []
                        for xidx,a in enumerate(oo):
                            for yidx,b in enumerate(a):
                                if b==True:
                                    mask.append(xidx)
                                    mask.append(yidx)
                        templist.append(mask)
                else:
                    mask = []
                    for xidx,a in enumerate(o):
                        for yidx,b in enumerate(a):
                            if b==True:
                                mask.append(xidx)
                                mask.append(yidx)
                    templist.append(mask)
                tempanns[idx]['segmentation'] = templist
                tempanns[idx]['area'] = len(templist[0])
            answers.extend(tempanns)
    return answers

In [ ]:
for i, out in tqdm_notebook(enumerate(output),total=len(output)):
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    
    img = make_img(image_info['file_name'])
    
    img['id'] = i + 2617
    
    images.append(img)
    
    ann = make_ann(img,out)
    
    anns.extend(ann)
    if i % 50 == 0:
        print(len(anns), len(images))

In [ ]:
with open('../../input/data/train.json') as f:
    info = json.load(f)

    for i in images:
        info['images'].append(i)
    for idx,an in enumerate(anns):
        an['id'] = idx + 21116
        info['annotations'].append(an)

    with open('./newtrain.json', 'w') as w:
        json.dump(info,w, ensure_ascii=False)
    print('done')